## 1.4 리딩 프레임
- 단백질의 번역 과정은 반드시 ATG 코돈에서 시작하기 때문에 이것을 찾아야만 어디가 번역의 시작점인지 알 수 있다.
- 하지만 이 코돈이 항상 서열의 시작점에 있는 것이 아니기 때문에 찾는 과정이 필요하다.
- ATG를 찾기 위해서 염기서열을 나누는 것을 리딩 프레임이라고 한다.
- 첫번째문자, 두번째문자, 세번째문자부터 시작하는 리딩 프레임이 존재하며, 이 서열이 주형가닥이 아닐때의 경우도 고려하여 하나의 염기 서열에 대해서 총 여섯개의 리딩 프레임이 존재한다.

### 1.4.1. 여섯개의 리딩프레임에서 DNA 서열을 계산
- DNA 서열을 아미노산으로 변경해주는 translate_seq() 함수를 이용하여 res 리스트에 아미노산 서열을 입력시킴
- 역상보서열을 만드는 reverse_complement() 함수를 이용하여 이 서열에 대한 아미노산 서열을 생성
- 아미노산 서열 res 반환

In [ ]:
def reading_frames(dna_seq):
    assert validate_dna(dna_seq), "Invalid DNA sequence"
    res = []
    #DNA서열
    res.append(translate_seq(dna_seq,0))
    res.append(translate_seq(dna_seq,1))
    res.append(translate_seq(dna_seq,2))
    #역상보서열
    rc = reverse_complement(dna_seq)
    res.append(translate_seq(rc,0))
    res.append(translate_seq(rc,1))
    res.append(translate_seq(rc,2))
    return res

❗ 역상보 서열을 사용한 이유는 원래 뒤에서부터 읽는 상보적인 서열을 앞에서부터 읽어오기 위함이다.  

### 1.4.2. 아미노산 서열에서 가능한 단백질 리스트 생성
- 오픈리딩 프레임(ORF) : 단백질로 번역될 가능성이 있는 서열
- 단백질 번역은 개시코돈에서 시작하여 종결코돈에서 끝나기 때문에 종결코돈의 유무를 확인해야함
- all_proteins_rf() 함수 생성
- DNA 서열이 아닌 아미노산 서열을 받아서 오픈리딩프레임 확인
- 단백질 서열 리스트 proteins 반환


In [ ]:
def all_proteins_rf(aa_seq):
    aa_seq = aa_seq.upper()
    current_prot = []
    proteins = []
    
    for aa in aa_seq:                                          #1 아미노산서열(aa_seq)의 아미노산(aa)에 대하여 
        if aa == "_":                                          #6 아미노산이 _이면
            if current_prot:                                   #7 current_prot에 이미 아미노산(aa)이 있다면
                for p in current_prot:                        
                    proteins.append(p)                         #8 aa string을 proteins에 append하고
                    current_prot = []                          #9 current_prot을 비움
        else:
            if aa == "M":                                      #2 아미노산(aa)이 M이면
                current_prot.append("")                        #3 current_prot에 문자열 자리 생성
            for i in range(len(current_prot)):                 #4 current_prot의 자리에
                current_prot[i] += aa                          #5 아미노산(aa) 추가
    
    return proteins

### 1.4.3. 모든 오픈 리딩프레임에서 가능한 단백질 계산
- reading_frames() : DNA 서열을 받아서 가능한 아미노산 서열을 반환 (6개)
- all_proteins_rf() : 아미노산 서열을 받아서 종결코돈을 고려한 단백질을 반환
- all_orfs() 함수 생성

In [ ]:
def all_orfs(dna_seq):
    assert validate_dna(dna_seq), "Invalid DNA sequence"
    rfs = reading_frames(dna_seq)                               #1 rfs - 아미노산 서열
    res = []
    for rf in rfs:                                              #2 모든 ORF에 대하여
        prots = all_proteins_rf(rf)                             #3 prots - 단백질 리스트
        for p in prots:
            res.append(p)                                       #4 res에 prots 원소들 append
    return res

### 1.4.4. 가능한 단백질을 최소 크기로 거르기
- 실제로는 단백질 크기가 작을수록 발현할 가능성이 크기 때문에 이를 고려해주면 좋음
- all_orfs_ord() 함수 생성 - DNA 서열과 최소 크기를 인수로 받아 단백질 서열을 크기별로 정렬해주는 함수

In [ ]:
#정렬 삽입 함수
def all_orfs_ord(dna_seq, minsize = 0):
    assert validate_dna(dna_seq), "Invalid DNA sequence"
    rfs = reading_frames(dna_seq)
    res = []
    for rf in rfs:
        prots = all_proteins_rf(rf)
        for p in prots:
            if len(p) > minsize:                                          #1 단백질의 크기가 최소크기보다 크면
                insert_prot_ord(p,res)                                    #2 insert_prot_ord() 연산 진행
    return res                                                                           

#정렬함수
def insert_prot_ord(prot, list_prots):
    i = 0                                                                  #3 i = 0 초기화
    while i < len(list_prots) and len(prot) < len(list_prots[i]):          #4 기존 i 인덱스의 단백질 크기가 더 크면
        i += 1                                                             #5 인덱스 값 i 에 1씩 더함 - 내림차순 정렬
    list_prots.insert(i, prot)                                             #6 최종 i 위치에 정렬 / 삽입